### From a File


In [ ]:
from main import create_taxonomy, add_categories

filename = "cais_data.csv"
brand = "CAIS"

taxonomy, df, samples = create_taxonomy(filename,
                                        text_column = "keyword",
                                        search_volume_column = "search_volume",
                                        platform = "openai", # "palm" or "openai"
                                        days = 30,
                                        S=500,
                                        ngram_range = (1, 6),
                                        min_df = 2,
                                        brand = None)

df = add_categories(taxonomy, df, brand) 

df.to_csv("cais_taxonomy.csv", index=False)

df.head()

### From a GSC Account

In [ ]:
from main import create_taxonomy

brand = "Green Group"

taxonomy, df, samples = create_taxonomy("sc-domain:greengroupcompanies.com",
                                        text_column = None,
                                        search_volume_column = None,
                                        platform = "openai", # "palm" or "openai"
                                        days = 30,
                                        S=100,
                                        ngram_range = (1, 6),
                                        min_df = 2,
                                        brand = None,
                                        limit_queries = 5)

df = add_categories(taxonomy, df, brand) 

df.to_csv("greengroupcompanies_taxonomy.csv", index=False)

df.head()